In [ ]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Utility Functions

These functions are copy pasted from the last three lectures, and implement the linear regression formulas:

In [ ]:
def standard_units(arr):
    """ Converts an array to standard units """
    return (arr - np.average(arr))/np.std(arr)

def correlation(t, x, y):
    """ Computes correlation: t is a table, and x and y are column names """
    x_standard = standard_units(t.column(x))
    y_standard = standard_units(t.column(y))
    return np.average(x_standard * y_standard)

def slope(t, x, y):
    """ Computes the slope of the regression line, like correlation above """
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

def intercept(t, x, y):
    """ Computes the intercept of the regression line, like slope above """
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean

def fitted_values(t, x, y):
    """Return an array of the regression estimates (predictions) at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

In [ ]:
# Some functions for plotting. You don't have to understand how any
# of the functions in this cell work, since they use things we 
# haven't learned about in Data 8.


def resize_window(lim=3.5):
    plots.xlim(-lim, lim)
    plots.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope + intercept
    plots.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plots.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_scatter(r):
    """Generate a scatter plot with a correlation approximately r"""
    plots.figure(figsize=(5,5))
    x, y = make_correlated_data(r)
    plots.scatter(x, y, color='darkblue', s=20)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)
    
def r_table(r):
    """
    Generate a table of 1000 data points with a correlation approximately r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

def show_demographics_rmse(slope, intercept):
    demographics_errors(slope, intercept)
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    prediction = slope * x + intercept
    mse = np.mean((y - prediction) ** 2)
    print("Root mean squared error:", round(mse ** 0.5, 2))

## 2016 election dataset

In [ ]:
demographics = Table.read_table('district_demographics2016.csv')
demographics

In [ ]:
predict_voting = demographics.select('Median Income', 'Percent voting for Clinton')
predict_voting = predict_voting.with_columns('Fitted',
    fitted_values(demographics, 'Median Income', 'Percent voting for Clinton'))

In [ ]:
predict_voting.scatter('Median Income')

In [ ]:
predict_income = demographics.select('College%', 'Median Income')
predict_income = predict_income.with_columns('Fitted',
    fitted_values(demographics, 'College%', 'Median Income'))
predict_income.scatter('College%')

## Nonlinear Regression ##

You can also use numerical optimization to make predictions when there is a nonlinear association among the data, if you have an idea for the nature of the association.

In [ ]:
shotput = Table.read_table('shotput.csv')
shotput

In [ ]:
shotput.scatter('Weight Lifted')

We can fit a linear regression model to this data.

In [ ]:
def shotput_linear_rmse(any_slope, any_intercept):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = any_slope*x + any_intercept
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
best_line = minimize(shotput_linear_rmse)
best_line

In [ ]:
weights = shotput.column(0)

In [ ]:
linear_fit = best_line.item(0)*weights + best_line.item(1)

shotput.with_column(
    'Best Line', linear_fit
).scatter(0)

You can see the predictions are systematically too low, on the left end.  The study that gathered this data hypothesized that there should be a quadratic relationship between weight lighted and shot put distance, so instead of fitting a line, let's fit a parabola -- a quadratic equation. 

A **quadratic Function** is given by the equation

$$
f(x) ~=~ ax^2 + bx + c
$$
for constants $a$, $b$, and $c$.  Let's try to find constants $a,b,c$ that yield a good fit to the data.

In [ ]:
def shotput_quadratic_rmse(a, b, c):
    x = shotput.column('Weight Lifted')
    y = shotput.column('Shot Put Distance')
    estimate = a*(x**2) + b*x + c
    return np.mean((y - estimate) ** 2) ** 0.5

In [ ]:
best_quad = minimize(shotput_quadratic_rmse)
best_quad

In [ ]:
# x = weight lifted = 100 kg
# Then predicted shot put distance:

(-0.00104)*(100**2) + 0.2827*100 - 1.5318

In [ ]:
quad_fit = best_quad.item(0)*(weights**2) + best_quad.item(1)*weights + best_quad.item(2)

In [ ]:
shotput.with_column('Best Quadratic Curve', quad_fit).scatter(0)

## Residuals

In [ ]:
def residuals(t, x, y):
    predictions = fitted_values(t, x, y)
    return t.column(y) - predictions # actual value - predicted value

In [ ]:
demographics = demographics.drop("State", "District", "Percent voting for Clinton").with_columns(
    'Fitted Value', fitted_values(demographics, 'College%', 'Median Income'),
    'Residual', residuals(demographics, 'College%', 'Median Income')
)
demographics

In [ ]:
demographics.scatter('College%')

In [ ]:
def plot_residuals(t, x, y):
    tbl = t.with_columns(
        'Fitted', fitted_values(t, x, y),
        'Residual', residuals(t, x, y)
    )
    tbl.select(x, y, 'Fitted').scatter(0)
    tbl.scatter(x, 'Residual')

In [ ]:
plot_residuals(demographics, 'College%', 'Median Income')

## Dugongs ##

In [ ]:
dugong = Table.read_table('dugong.csv')
dugong.show(5)

In [ ]:
dugong.scatter('Length', 'Age')

In [ ]:
correlation(dugong, 'Length', 'Age')

At a glance, we have a **very** strong correlation - which might indicate that linear regression is a good idea! However, let's take a look at the residuals first.

In [ ]:
plot_residuals(dugong, 'Length', 'Age')

Seeing this, you should probably be a little sceptical. We see a **pattern** in the residuals. We're consistently overestimating in some areas, and underestimating in others. This indicates that we could probably find a model that's better than a linear one. 

# US Women

As an example of another residual plot, let's look at the relationship between height and average weight for Women's Division Athletes in the US.

In [ ]:
us_women = Table.read_table('us_women.csv')
us_women.show(5)

In [ ]:
us_women.scatter('height')

In [ ]:
correlation(us_women, 'height', 'ave weight')

In [ ]:
plot_residuals(us_women, 'height', 'ave weight')

## Average of Residuals ##

Let's look at some nice properties of residuals, for the three datasets we talked about today.

1. The average of the residuals will always be 0

In [ ]:
round(np.average(residuals(dugong, 'Length', 'Age')), 6)

In [ ]:
round(np.average(residuals(us_women, 'height', 'ave weight')), 6)

In [ ]:
round(np.average(residuals(demographics, 'College%', 'Median Income')), 6)

2. The correlation between the residuals and the x values is 0.

In [ ]:
demographics.scatter('College%', 'Residual')
round(correlation(demographics, 'College%', 'Residual'), 6)

3. The correlation between the residuals and the predicted values is 0.

In [ ]:
demographics.scatter('Fitted Value', 'Residual')
round(correlation(demographics, 'Fitted Value', 'Residual'), 6)